# 風險地圖

In [1]:
import pandas as pd
import folium
import json
import geopandas
import folium
from folium.plugins import MarkerCluster, Search
from json2html import *
data = pd.read_csv("106年火災事件資料.csv")
data_car_A1 =  pd.read_csv("108年度A1類交通事故資料.csv")
data_car_A2 = pd.read_csv("108年度A1類交通事故資料.csv")

## 統計各區域火災數量 (106年)

In [2]:
data_modified = data[(data["縣市"]!="高雄港") & (data["縣市"]!="臺中港")]
df = pd.DataFrame()
df["區域"] = pd.Series(data_modified["鄉_鎮_市_區"].value_counts().index.values)
df["數量"] = pd.Series(data_modified["鄉_鎮_市_區"].value_counts().values)
df

,區域,數量
0,沙鹿區,401
1,南區,393
2,安南區,381
3,彰化市,350
4,中山區,347
...,...,...
350,茂林區,2
351,大同鄉,2
352,鹿谷鄉,1
353,泰武鄉,1


## 統計車禍發生地點 (107年)

In [12]:
data_car_pos_A1 = data_car_A1[["經度","緯度","車種","死亡受傷人數"]]
data_car_pos_A1 = data_car_A1.dropna(axis=0, how='any')
data_car_pos_A2 = data_car_A1[["經度","緯度","車種","死亡受傷人數"]]
data_car_pos_A2 = data_car_A1.dropna(axis=0, how='any')
for item in data_car_pos_A1.values.tolist()[:3]:
    print("車種:{}<br>死亡人數:{}".format(item[3],item[2]))

車種:普通重型-機車;自用-小貨車(含客、貨兩用)<br>死亡人數:死亡1;受傷0
車種:普通重型-機車<br>死亡人數:死亡1;受傷0
車種:大型重型1(550C.C.以上)-機車<br>死亡人數:死亡1;受傷0


In [5]:
from folium.plugins import MarkerCluster
Taiwan_car_map = folium.Map(location=[23.58,121], zoom_start=7)
marker_cluster = MarkerCluster().add_to(Taiwan_car_map)

for item in data_car_pos_A1.values.tolist():
    popuptext = "<h4>車種:</h4> {}<br><h4>死亡人數:</h4>{}".format(item[2],item[3])
    test = folium.Html(popuptext, script=True)
    popup = folium.Popup(test, max_width=300,min_width=300)
    folium.Marker(
        location=[item[5],item[4]],
        popup = popup,
        icon=None).add_to(marker_cluster)
Taiwan_car_map.save('Folium_車禍統計.html')    
#Taiwan_car_map

In [6]:
Taiwan_map = folium.Map(location=[23.58,121], zoom_start=7)
folium.Choropleth(
    geo_data="鄉鎮市區行政區域界線.json",
    name='choropleth',
    data = df,
    fill_color='YlGnBu',
    columns=["區域","數量"],
    key_on='feature.properties.T_Name',
).add_to(Taiwan_map)

#display(Taiwan_map)
Taiwan_map.save('Folium_火災統計.html')

In [15]:
Total_map = folium.Map(location=[23.58,121], zoom_start=7)
marker_cluster = MarkerCluster(name="A1車禍統計").add_to(Total_map)

for item in data_car_pos_A1.values.tolist():
    popuptext = "<h4>車種:</h4> {}<br><h4>死亡人數:</h4>{}".format(item[3],item[2])
    test = folium.Html(popuptext, script=True)
    popup = folium.Popup(test, max_width=300,min_width=300)
    folium.Marker(
        location=[item[5],item[4]],
        popup = popup,
        icon=folium.Icon(color='darkred', icon='car', prefix='fa')).add_to(marker_cluster)
    
marker_cluster = MarkerCluster(name="A2車禍統計").add_to(Total_map)

for item in data_car_pos_A2.values.tolist():
    popuptext = "<h4>車種:</h4> {}<br><h4>死亡人數:</h4>{}".format(item[3],item[2])
    test = folium.Html(popuptext, script=True)
    popup = folium.Popup(test, max_width=300,min_width=300)
    folium.Marker(
        location=[item[5],item[4]],
        popup = popup,
        icon=folium.Icon(color='darkred', icon='car', prefix='fa')).add_to(marker_cluster)
    
folium.Choropleth(
    geo_data="鄉鎮市區行政區域界線.json",
    name='火災事件',
    data = df,
    fill_color='YlGnBu',
    columns=["區域","數量"],
    key_on='feature.properties.T_Name',
).add_to(Total_map)

with open("彰化縣107_geo.json", "r") as f:
    JSON_file = json.load(f)
building= geopandas.read_file("彰化縣107_geo.json")
addressgeo = folium.GeoJson(
    JSON_file,
    name='建築位置',
    show=False
).add_to(Total_map)

statesearch = Search(
    layer=addressgeo,
    geom_type='Points',
    placeholder='搜尋地址',
    collapsed=False,
    search_label='地址',
    search_zoom=17,
    weight=3,
    #tooltip=folium.GeoJsonTooltip(fields=["地址"],aliases=["地址"])
).add_to(Total_map)

marker_cluster = MarkerCluster(name="建築標記").add_to(Total_map)
address_list=[]

for i in range(len(building)):
    try:
        popuptext = "<h4>{}</h4>{}".format(JSON_file["features"][i]["properties"]["地址"]
                                           ,json2html.convert(json = JSON_file["features"][i]["properties"]["樓層概要"]))
    except:
        popuptext = "<h4>{}</h4>".format(JSON_file["features"][i]["properties"]["地址"])
    test = folium.Html(popuptext, script=True)
    popup = folium.Popup(test, max_width=300,min_width=300)
    folium.Marker(
      popup = popup,
      location=[building["經緯"].values[i].split(",")[0],building["經緯"].values[i].split(",")[1]],
      icon=folium.Icon(color='darkgreen', icon='home', prefix='fa')).add_to(marker_cluster)
    
folium.LayerControl().add_to(Total_map)


Total_map.save('index.html')